In [6]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [67]:
df = pd.read_csv('C:/Users/lab/Desktop/7777.csv', encoding='gbk')
df = df.iloc[3:42, 2:8]
data = df.values
data2 = data[:, :5]
data2[:, 2] = data[:, 5]
data2

array([['61.00 ', '59.00 ', '93', '46.67 ', '55.56 '],
       ['71.00 ', '56.00 ', '86.4', '40.67 ', '55.89 '],
       ['61.00 ', '55.00 ', '88', '52.00 ', '56.00 '],
       ['63.00 ', '66.00 ', '92', '40.67 ', '56.56 '],
       ['72.00 ', '49.00 ', '82.4', '49.33 ', '56.78 '],
       ['61.00 ', '60.00 ', '83', '49.33 ', '56.78 '],
       ['65.00 ', '60.00 ', '80.2', '45.33 ', '56.78 '],
       ['66.00 ', '68.00 ', '90.00', '38.67 ', '57.56 '],
       ['60.00 ', '65.00 ', '80', '48.67 ', '57.89 '],
       ['68.00 ', '56.00 ', '86.80', '50.67 ', '58.22 '],
       ['61.00 ', '63.00 ', '82.4', '51.33 ', '58.44 '],
       ['75.00 ', '49.00 ', '82', '53.33 ', '59.11 '],
       ['67.00 ', '70.00 ', '92.00', '40.67 ', '59.22 '],
       ['65.00 ', '64.00 ', '82.2', '49.33 ', '59.44 '],
       ['63.00 ', '64.00 ', '85.20', '56.67 ', '61.22 '],
       ['75.00 ', '56.00 ', '83.00', '52.67 ', '61.22 '],
       ['71.00 ', '70.00 ', '90', '43.33 ', '61.44 '],
       ['57.00 ', '75.00 ', '81', '53.33

实际上 $E = (A + B + D ) /3$，C是干扰项

In [69]:
split = int(len(data2)*0.7)
X = data[:split, :4].astype(np.float32)
Y = data[:split, 4].astype(np.float32)
X2 = data[split:, :4].astype(np.float32)
Y2 = data[split:, 4].astype(np.float32)
X, Y, X2, Y2

(array([[61.  , 59.  , 93.  , 46.67],
        [71.  , 56.  , 86.4 , 40.67],
        [61.  , 55.  , 88.  , 52.  ],
        [63.  , 66.  , 92.  , 40.67],
        [72.  , 49.  , 82.4 , 49.33],
        [61.  , 60.  , 83.  , 49.33],
        [65.  , 60.  , 80.2 , 45.33],
        [66.  , 68.  , 90.  , 38.67],
        [60.  , 65.  , 80.  , 48.67],
        [68.  , 56.  , 86.8 , 50.67],
        [61.  , 63.  , 82.4 , 51.33],
        [75.  , 49.  , 82.  , 53.33],
        [67.  , 70.  , 92.  , 40.67],
        [65.  , 64.  , 82.2 , 49.33],
        [63.  , 64.  , 85.2 , 56.67],
        [75.  , 56.  , 83.  , 52.67],
        [71.  , 70.  , 90.  , 43.33],
        [57.  , 75.  , 81.  , 53.33],
        [67.  , 68.  , 84.2 , 52.  ],
        [66.  , 83.  , 91.  , 38.67],
        [69.  , 62.  , 85.2 , 56.67],
        [69.  , 68.  , 79.2 , 54.67],
        [64.  , 57.  , 88.6 , 71.33],
        [71.  , 61.  , 81.  , 61.33],
        [63.  , 77.  , 89.6 , 53.33],
        [67.  , 68.  , 93.  , 58.67],
        [72.

```python
self.Y = torch.tensor(Y).unsqueeze(1)
```
假设输入特征维度是4，输出标签的特征维度是1，A取`batch_size`个输入到网络得到shape是`[batch_size, 4] -> [batch_size, 1]`，而不是`[batch_size]`。所以，你制作数据集的label时，应该是`[n, 1]`的样子。

In [81]:
class MyDataset(Dataset):
    def __init__(self, X, Y=None):
        """
        Args:
            @param Y: When Y=none, the data set is "test". Otherwise, it is the training set
        """
        super(MyDataset, self).__init__()
        self.X = torch.tensor(X)                # [n, 4]
        if Y is not None:
            assert len(X) == len(Y)
            self.Y = torch.tensor(Y).unsqueeze(1)   # [n, 1]
        else:
            self.Y = None
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        if self.Y is not None:
            return self.X[index], self.Y[index]
        else:
            return self.X[index]

In [82]:
train_dataset = MyDataset(X, Y)
train_dl = DataLoader(train_dataset, batch_size=1)

test_dataset = MyDataset(X2)
test_dl = DataLoader(test_dataset, batch_size=1)

In [108]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 1)
        )
    
    def forward(self, X):
        out = self.linear(X)
        return out

device = torch.device('cuda')
net = Net().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss = nn.MSELoss()

def init_param(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
        
net.apply(init_param)

Net(
  (linear): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [ ]:
epoch = 0
while epoch < 50000:
    for x,y in train_dl:
        optimizer.zero_grad()
        x,y = x.to(device),y.to(device)
        y_hat = net(x)
        # print(y, y_hat)
        l = loss(y, y_hat)
        l.backward()
        optimizer.step()
        print(f'{epoch} : {l.item()}')
        epoch += 1

In [114]:
net.eval()
y_preds = []
for x in test_dl:
    x = x.to(device)
    with torch.no_grad():
        y_pred = net(x)
    y_preds.append(y_pred.detach().item())

In [117]:
np.abs(y_preds - Y2) 

array([0.07260132, 0.10398102, 0.09038544, 0.06794739, 0.14838409,
       0.15836334, 0.14992523, 0.16079712, 0.08699036, 0.19973755,
       0.14920044, 0.2219696 ])

In [116]:
y_preds, Y2

([64.92739868164062,
  64.8960189819336,
  65.46961212158203,
  66.15205383300781,
  66.6316146850586,
  66.73163604736328,
  67.74007415771484,
  68.72920227050781,
  70.02301025390625,
  71.58026123046875,
  72.18080139160156,
  74.77803039550781],
 array([65.  , 65.  , 65.56, 66.22, 66.78, 66.89, 67.89, 68.89, 70.11,
        71.78, 72.33, 75.  ], dtype=float32))